In [1]:
import sqlite3
import pandas as pd
import numpy as np

# Entity Relationship Diagram

<img src="images/erd.png">

# Create Tables

In [2]:
conn = sqlite3.connect("../data/db.sqlite3")
cur = conn.cursor()

In [3]:
drop_table_queries = ['research_profile', 'author', 'school',
                      'sdg', 'neda', 'journal', 'budget_source',
                      'funding_agency', 'funding_type', 'research_status', 
                      'utilization', 'patent']

drop_table_queries = list(map(lambda x: f'DROP TABLE IF EXISTS {x};', 
                              drop_table_queries))

def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

drop_tables(cur, conn)

In [4]:
author_create = ("""
CREATE TABLE author (
    author_id INTEGER PRIMARY KEY AUTOINCREMENT,
    author_name VARCHAR NOT NULL UNIQUE 
)
""")

author_research_create = ("""
CREATE TABLE author_research (
    research_id INTEGER,
    author_id INTEGER 
)
""")

sdg_create = ("""
CREATE TABLE sdg (
    sdg_id INTEGER PRIMARY KEY AUTOINCREMENT,
    sdg_name VARCHAR NOT NULL UNIQUE
)
""")

neda_create = ("""
CREATE TABLE neda (
    neda_id INTEGER PRIMARY KEY AUTOINCREMENT,
    neda_name VARCHAR NOT NULL UNIQUE 
)
""")

journal_create = ("""
CREATE TABLE journal (
    journal_id INTEGER PRIMARY KEY AUTOINCREMENT,
    journal_name VARCHAR NOT NULL,
    volume_no VARCHAR,
    issue_no VARCHAR,
    year_published INTEGER,
    no_citations INTEGER,
    scope VARCHAR
)
""")

rs_create = ("""
CREATE TABLE research_status (
    rs_id INTEGER PRIMARY KEY AUTOINCREMENT,
    rs_name VARCHAR NOT NULL UNIQUE
)
""")

school_create = ("""
CREATE TABLE school (
    school_id INTEGER PRIMARY KEY AUTOINCREMENT,
    school_name VARCHAR NOT NULL,
    school_abbreviation VARCHAR,
    campus VARCHAR, 
    city VARCHAR, 
    province VARCHAR, 
    region VARCHAR
)
""")

bs_create = ("""
CREATE TABLE budget_source (
    bs_id INTEGER PRIMARY KEY AUTOINCREMENT,
    bs_name VARCHAR NOT NULL UNIQUE
)
""")

fa_create = ("""
CREATE TABLE funding_agency (
    fa_id INTEGER PRIMARY KEY AUTOINCREMENT,
    fa_name VARCHAR NOT NULL UNIQUE 
)
""")

ft_create = ("""
CREATE TABLE funding_type (
    ft_id INTEGER PRIMARY KEY AUTOINCREMENT,
    ft_name VARCHAR NOT NULL UNIQUE 
)
""")

research_profile_create = ("""
CREATE TABLE research_profile (
    research_id INTEGER PRIMARY KEY AUTOINCREMENT,
    research_title VARCHAR NOT NULL, 
    author_id INTEGER NOT NULL, 
    keywords VARCHAR,
    abstract VARCHAR, 
    year INTEGER, 
    school_id INTEGER, 
    sdg_id INTEGER,
    neda_id INTEGER, 
    journal_id INTEGER, 
    allocated_budget INTEGER, 
    bs_id INTEGER, 
    fa_id INTEGER, 
    ft_id INTEGER, 
    rs_id INTEGER,
    CONSTRAINT fk_author 
        FOREIGN KEY (author_id)
        REFERENCES author(author_id), 
    CONSTRAINT fk_sdg
        FOREIGN KEY (sdg_id)
        REFERENCES sdg(sdg_id),
    CONSTRAINT fk_neda
        FOREIGN KEY (neda_id)
        REFERENCES neda(neda_id),
    CONSTRAINT fk_journal
        FOREIGN KEY (journal_id)
        REFERENCES journal(journal_id),
    CONSTRAINT fk_school
        FOREIGN KEY (school_id)
        REFERENCES school(school_id),
    CONSTRAINT fk_bs
        FOREIGN KEY (bs_id)
        REFERENCES budget_source(bs_id),
    CONSTRAINT fk_fa
        FOREIGN KEY (fa_id)
        REFERENCES funding_agency(fa_id),
    CONSTRAINT fk_ft
        FOREIGN KEY (ft_id)
        REFERENCES funding_type(ft_id),
    CONSTRAINT fk_rs
        FOREIGN KEY (rs_id)
        REFERENCES research_status(rs_id)
    
)
""")

utilization_create = ("""
CREATE TABLE utilization (
    util_id INTEGER PRIMARY KEY AUTOINCREMENT,
    research_title VARCHAR NOT NULL, 
    product_service VARCHAR, 
    simplified VARCHAR,
    beneficiary VARCHAR, 
    school_id INTEGER, 
    year INTEGER, 
    CONSTRAINT fk_school
        FOREIGN KEY (school_id)
        REFERENCES school(school_id)
)
""")

patent_create = ("""
CREATE TABLE patent (
    patent_id INTEGER PRIMARY KEY AUTOINCREMENT,
    patent_filed VARCHAR NOT NULL, 
    patent_type VARCHAR, 
    patent_status VARCHAR,
    date_register VARCHAR, 
    registration_number INTEGER, 
    author_id INTEGER,
    school_id INTEGER,
    CONSTRAINT fk_author
        FOREIGN KEY (author_id)
        REFERENCES author(author_id),
    CONSTRAINT fk_school
        FOREIGN KEY (school_id)
        REFERENCES school(school_id)
)
""")

create_table_queries = [author_create, sdg_create, 
                        neda_create, journal_create, 
                        rs_create, school_create, 
                        bs_create, fa_create, ft_create,  
                        research_profile_create,
                        utilization_create, patent_create]

def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()
    
create_tables(cur, conn)

# ETL

### Research Profile

In [5]:
res_prof = pd.read_excel('../data/cleaned/research_profile.xlsx')
display(res_prof.iloc[:, 2:-1].head())

,Research Title,Author,Keywords,Abstract,Year (YYYY),University (Full Name),University (Abbreviation),Campus,City/Municipality,Province,...,Volume No,Issue No,Year Published,Number of Citations as of Jan 2022,Local/International,"Allocated Budget (PH Pesos) 1,000,000.00",Source,Funding Agency,"Type\n(Grant, Commissioned)",Status \n(Completed/Ongoing)
0,Disaster Risk Reduction and Long-Term Risk Res...,"Narisma, Gemma T.",disaster risk reduction and long-term risk res...,The CHED PHERNet-AdMU Program has been conduct...,2017.0,Ateneo de Manila University,ADMU,Main,Quezon City,Quezon city,...,NaN,NaN,NaN,NaN,NaN,10000000,External,CHED,Grant,Completed
1,Antimycotic Activity Of Nostoc Linckia Specie...,"Paderes, Nero M.",microbiology;anti-staphylococcus aureus;plant ...,The warm tropical climate of the Philippines a...,2018.0,Abra State Institute of Science and Technology,ASIST,Main,Lagangilang,Abra,...,24.0,NaN,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,Completed
2,Green Chemistry Perspective Among Stem 12 Seni...,"Paderes, Nero M.",green chemistry;attitude;practices;knowledge;s...,The study was conducted to determine the “Gree...,2018.0,Abra State Institute of Science and Technology,ASIST,Main,Lagangilang,Abra,...,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Completed
3,Solid Waste Management System of Barangay Deet,"Bose, Ma. Digna T.",NaN,This study aimed to establish a Solid Waste Ma...,2019.0,Abra State Institute of Science and Technology,ASIST,Main,Lagangilang,Abra,...,25.0,NaN,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,Completed
4,Multivariate Analysis on Tobacco Farming System,"Gannapao, Jubert S.;Guidang, Excel Philip B.;T...",NaN,Farming systems refers to the utilization of t...,2018.0,Abra State Institute of Science and Technology,ASIST,Main,Lagangilang,Abra,...,27.0,NaN,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,Completed


In [6]:
def check_author_similarity(unique_authors):
    one_step_authors = unique_authors[1:]
    replace_authors = {}
    for author, step_author in zip(unique_authors, one_step_authors):
        count = 0
        for (name_author, 
             name_step) in zip(list(map(lambda x: x.strip(), author.split())),
                           list(map(lambda x: x.strip(), step_author.split()))
                            ):

            if name_author == name_step:
                count += 1

        if count >= 2:
            if len(author) < len(step_author):
                replace_authors[step_author] = author
            else:
                replace_authors[author] = step_author
    
    return replace_authors
    

In [7]:
author_insert = """
INSERT INTO author (author_name)
VALUES (?)
"""

def process_author(cur, conn, res_prof):
    unique_authors = sorted(res_prof['Author'].str.split(';')
                                              .explode().unique())
    unique_authors.remove('')
    unique_authors.remove(',')
    while check_author_similarity(unique_authors):
        for key, item in check_author_similarity(unique_authors).items():
            res_prof['Author'] = (res_prof['Author'].str.replace(key, item, 
                                                                regex=False)
                                           .apply(lambda x: x if x[-1] != ';' 
                                                  else x[:-1])
                                           .str.replace(';;', ';',
                                                        regex=False))
                            
            unique_authors = sorted(res_prof['Author']
                                    .str.split(';')
                                    .explode().unique())
            try:
                unique_authors.remove('')
                unique_authors.remove(',')
            except:
                pass
    
    
    for author in unique_authors:
        cur.execute(author_insert, tuple([author]))
        
    conn.commit()
    
    res_prof['Author'] = res_prof['Author'].str.split(';')
    res_prof = res_prof.explode('Author')
    cur.execute('SELECT * FROM author;')
    author_table = pd.DataFrame(cur.fetchall(), columns=['author_id',
                                                         'author'])
    res_prof = pd.merge(res_prof, author_table, 
                        left_on='Author', 
                        right_on='author', 
                        how='left')

    res_prof = res_prof.drop(['author', 'Author'], axis=1)
    
    return res_prof
    
res_prof = process_author(cur, conn, res_prof)
    

In [8]:
sdg_insert = """
INSERT INTO sdg (sdg_id, sdg_name)
VALUES (?, ?)
"""

def process_sdg(cur, conn, res_prof):
    sdg_mapping = [
        [1, 'No poverty'],
        [2, 'Zero hunger'],
        [3, 'Good health and well-being'],
        [4, 'Quality education'],
        [5, 'Gender equality'],
        [6, 'Clean water and sanitation'],
        [7, 'Affordable and clean energy'],
        [8, 'Decent work and economic growth'],
        [9, 'Industry, Innovation and Infrastructure'],
        [10, 'Reduced inequality'],
        [11, 'Sustainable cities and communities'],
        [12, 'Responsible consumption and production'],
        [13, 'Climate action'],
        [14, 'Life below water'],
        [15, 'Life on land']
    ]
    
    cur.executemany(sdg_insert, sdg_mapping)
    conn.commit()
    
    res_prof['SDG'] = res_prof['SDG'].astype(str).str.split('; ')
    res_prof = res_prof.explode('SDG')
    res_prof.loc[res_prof['SDG'] == 'nan', 'SDG'] = np.nan
    
    return res_prof

res_prof = process_sdg(cur, conn, res_prof)

In [9]:
neda_insert = """
INSERT INTO neda (neda_id, neda_name)
VALUES (?, ?)
"""

def process_neda(cur, conn):
    neda_mapping = [
        [1, '"Malasakit" Enhancing the Social Fabric'],
        [2, '"Pagbabago" Reducing Inequality'],
        [3, '"Patuloy na Pag-unlad" Increasing Growth Potential']
    ]
    
    cur.executemany(neda_insert, neda_mapping)
    conn.commit()
    
process_neda(cur, conn)

In [10]:
journal_insert = """
INSERT INTO journal (journal_name, volume_no, issue_no, year_published, 
                     no_citations, scope)
VALUES (?, ?, ?, ?, ?, ?)
"""

def process_journals(cur, conn, res_prof):
    journals = res_prof.loc[:, 'Journal Title':'Local/International']
    journals = journals[~journals['Journal Title'].isnull()].drop_duplicates()
    journals = journals.values.tolist()
    cur.executemany(journal_insert, journals)
    conn.commit()

    cols = res_prof.loc[:, 'Journal Title':'Local/International'].columns
    res_prof['Volume No'] = res_prof['Volume No'].astype(str) 
    cur.execute('SELECT * FROM journal;')
    journal_table = pd.DataFrame(cur.fetchall(), columns=['journal_id'] 
                                 + cols.tolist())
    
    res_prof = pd.merge(res_prof, journal_table, 
                        left_on=cols.tolist(), 
                        right_on=cols.tolist(), 
                        how='left')
    res_prof = res_prof.drop(cols, axis=1)
    return res_prof

res_prof = process_journals(cur, conn, res_prof)

In [11]:
res_stat_insert = """
INSERT INTO research_status (rs_id, rs_name)
VALUES (?, ?)
"""
def process_research_status(cur, conn, res_prof):
    res_stat_mapping = [
        [0, 'Ongoing'],
        [1, 'Completed']
    ]
    cur.executemany(res_stat_insert, res_stat_mapping)
    conn.commit()
    col = 'Status \n(Completed/Ongoing)'
    res_prof[col] = res_prof[col].str.replace('completed', 'Completed')
    res_stat_table = pd.DataFrame(res_stat_mapping, columns=['rs_id', col])
    res_prof = pd.merge(res_prof, res_stat_table, 
                        left_on=col, 
                        right_on=col, 
                        how='left')
    res_prof = res_prof.drop(col, axis=1)
    
    return res_prof
    
res_prof = process_research_status(cur, conn, res_prof)

In [12]:
budget_source_insert = """
INSERT INTO budget_source (bs_id, bs_name)
VALUES (?, ?)
"""

def process_budget_source(cur, conn, res_prof):
    bs_mapping = [
        [0, 'Internal'],
        [1, 'External']
    ]
    cur.executemany(budget_source_insert, bs_mapping)
    conn.commit()
    
    col = 'Source'
    res_prof[col] = res_prof[col].str.split('; ')
    res_prof = res_prof.explode(col)
    bs_table = pd.DataFrame(bs_mapping, columns=['bs_id', col])
    res_prof = pd.merge(res_prof, bs_table, 
                        left_on=col, 
                        right_on=col, 
                        how='left')
    res_prof = res_prof.drop(col, axis=1)
    return res_prof

res_prof = process_budget_source(cur, conn, res_prof)

In [13]:
funding_agency_insert = """
INSERT INTO funding_agency (fa_name)
VALUES (?)
"""

def process_funding_agency(cur, conn, res_prof):
    fa_mapping = sorted(res_prof['Funding Agency'].dropna().unique())
    for fa in fa_mapping:
        cur.execute(funding_agency_insert, tuple([fa]))
    conn.commit()
    
    cur.execute('SELECT * FROM funding_agency;')
    fa_table = pd.DataFrame(cur.fetchall(), columns=['fa_id',
                                                    'Funding Agency'])
    res_prof = pd.merge(res_prof, fa_table, 
                    left_on='Funding Agency', 
                    right_on='Funding Agency', 
                    how='left')
    
    res_prof = res_prof.drop('Funding Agency', axis=1)
    
    return res_prof

res_prof = process_funding_agency(cur, conn, res_prof)

In [14]:
funding_type_insert = """
INSERT INTO funding_type (ft_name)
VALUES (?)
"""

def process_funding_type(cur, conn, res_prof):
    ft_mapping = sorted(res_prof['Type\n(Grant, Commissioned)'].dropna()
                                                               .unique())
    for ft in ft_mapping:
        cur.execute(funding_type_insert, tuple([ft]))
    conn.commit()
    
    cur.execute('SELECT * FROM funding_type;')
    ft_table = pd.DataFrame(cur.fetchall(), 
                            columns=['ft_id',
                                     'Type\n(Grant, Commissioned)'])
    res_prof = pd.merge(res_prof, ft_table, 
                    left_on='Type\n(Grant, Commissioned)', 
                    right_on='Type\n(Grant, Commissioned)', 
                    how='left')
    res_prof = res_prof.drop('Type\n(Grant, Commissioned)', axis=1)
    
    return res_prof

res_prof = process_funding_type(cur, conn, res_prof)

In [15]:
school_insert = """
INSERT INTO school (school_name, school_abbreviation, 
                    campus, city, province, region)
VALUES (?, ?, ?, ?, ?, ?)
"""

def process_school(cur, conn, res_prof):
    schools = pd.read_csv('../data/cleaned/SUC_Info.csv')
    sch = schools.loc[:, 'University (Full Name)':'Region'].copy()
    cols = sch.columns
    for col in cols:
        sch[col] = sch[col].astype(str).str.strip()
    sch = (sch.drop_duplicates()
              .sort_values(by='University (Full Name)'))
    cur.executemany(school_insert, sch.values.tolist())
    conn.commit()
    
    cur.execute('SELECT * FROM school;')
    school_table = pd.DataFrame(cur.fetchall(), columns=['school_id'] 
                             + cols.tolist()).copy()
    
    for col in cols:
        school_table[col] = school_table[col].astype(str).str.strip()
    
    cols = cols.tolist()
    cols.remove('Region')
    res_prof = pd.merge(res_prof.drop('Region',axis=1), school_table, 
                    left_on=cols, 
                    right_on=cols, 
                    how='left')
    
    res_prof = res_prof.drop(cols + ['Region'], axis=1)

    return res_prof
    
res_prof = process_school(cur, conn, res_prof)


In [16]:
research_profile_insert = """
INSERT INTO research_profile (research_title, author_id, keywords, 
                    abstract, year, school_id, sdg_id,
                    neda_id, journal_id, allocated_budget,
                    bs_id, fa_id, ft_id, rs_id)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

def process_research_profile(cur, conn, res_prof):
    res_prof = res_prof.loc[:, ['Research Title', 'author_id', 'Keywords', 
                'Abstract', 'Year (YYYY)', 'school_id', 'SDG', 
                'NEDA Priority Area', 'journal_id', 
                'Allocated Budget (PH Pesos) 1,000,000.00',
                'bs_id', 'fa_id', 'ft_id', 'rs_id']]
    
    cur.executemany(research_profile_insert, 
                    res_prof.values.tolist())
    
    conn.commit()
    
process_research_profile(cur, conn, res_prof)

### Utilization

In [17]:
# load data from source
utils = pd.read_excel('../data/cleaned/utilization.xlsx')
display(utils.iloc[:, 2:].head())

,Research Title,Product/Service,Simplified,Beneficiary,University,Campus,Year
0,Traditional Rice Varieties (Aromatic Rice): Gr...,Technology In Growing Traditional Rice,Rice Growing Technology,Francisca M. Badua,Agusan del Sur State College of Agriculture an...,Main,2018
1,Agricultural Wastes As Potential Substrates Fo...,Growth Media From Organic Waste,Growth Medium,Loreta B. Alayon; Flora Anisco; Felicidad D. A...,Agusan del Sur State College of Agriculture an...,Main,2018
2,The Response Of Tissue Cultured Banana (Musa S...,Fertilizer For Banana,Fertilizer,Farmers Association of lower Agpan; Imelda Tec...,Agusan del Sur State College of Agriculture an...,Main,2018
3,Manobo Translator Mobile Application,Mobile Application,Mobile Application,Indigent People,Agusan del Sur State College of Agriculture an...,Main,2019
4,Development Of Isulat: A Mobile Instructional ...,Mobile Application,Mobile Application,Teachers,Agusan del Sur State College of Agriculture an...,Main,2019


In [18]:
# data cleaning
utils.loc[((utils.University == 'University of the Philippines') & 
          (utils.Campus == 'Baguio')), 
          'University'] = 'University of the Philippines Baguio'


utils.loc[((utils.University == 'Tarlac State University') & 
          (utils.Campus == 'San Isidro')), 
          'Campus'] = 'San isidro'

In [19]:
utilization_insert = """
INSERT INTO utilization (research_title, product_service, simplified, 
                         beneficiary, school_id, year)
VALUES (?, ?, ?, ?, ?, ?)
"""

def process_utilization(cur, conn, utils):
    cur.execute('SELECT * FROM school;')
    schools = pd.DataFrame(cur.fetchall(), 
                           columns=['school_id', 'school_name',
                                    'school_abbreviation', 'campus',
                                    'city', 'province', 'region'])

    merge = pd.merge(utils, schools, left_on=['University', 'Campus'], 
                     right_on=['school_name', 'campus'], 
                     how='left')
    merge = merge.loc[:, ['Research Title', 'Product/Service', 
                          'Simplified', 'Beneficiary', 'school_id', 
                          'Year']]
    
    cur.executemany(utilization_insert, 
                    merge.values.tolist())
    conn.commit()
    
process_utilization(cur, conn, utils)

### Patents

In [20]:
# load data from source
patents = pd.read_excel('../data/cleaned/patents.xlsx')
display(patents.iloc[:, 3:].head())

,Patent Filed,Patent Type,Patent Status,Date of Registration,Registration Number,Author,University
0,Radio Frequency-Based Mobile Charger,Utility Model,Registered,2020-04-22,2-2018-000289,"Lomboy, Orlando A.",Abra State Institute of Sciences and Technology
1,Process Of Producing Molluscide Mixture From E...,Utility Model,Registered,2020-10-02,2-2018-000326,"Venus, Victoria G.",Abra State Institute of Sciences and Technology
2,Process Of Producing Tiessa (Pouteria Campechi...,Utility Model,Registered,2020-08-07,2-2019-001395,"Gonzalo, Gemma P.",Abra State Institute of Sciences and Technology
3,Process Of Constructing An Educational Mathema...,Utility Model,Registered,2020-02-19,2-2019-001396,"Atmosfera, Rynheart P.",Abra State Institute of Sciences and Technology
4,Process Of Constructing An Educational Mathema...,Utility Model,Registered,2020-02-19,2-2019-001397,"Atmosfera, Rynheart P.",Abra State Institute of Sciences and Technology


In [21]:
author_insert = """
INSERT INTO author (author_name)
VALUES (?)
"""

def process_patent_author(cur, conn, patents):
    authors = (patents.Author.str.split(';')
                      .explode()
                      .str.strip()
                      .drop_duplicates().tolist())
    
    cur.execute('SELECT * FROM author')
    author_table = cur.fetchall()
    author_table = [i[1] for i in author_table]
    
    unique_authors = sorted(set(authors) - set(author_table))
    
    for author in unique_authors:
        cur.execute(author_insert, tuple([author]))
        
    conn.commit()
    
    patents['Author'] = patents['Author'].str.split(';')
    patents = patents.explode('Author')
    cur.execute('SELECT * FROM author;')
    author_table = pd.DataFrame(cur.fetchall(), columns=['author_id',
                                                         'author'])
    patents = pd.merge(patents, author_table, 
                        left_on='Author', 
                        right_on='author', 
                        how='left')

    patents = patents.drop(['author', 'Author'], axis=1)
    
    return patents
    
patents = process_patent_author(cur, conn, patents)
    

In [22]:
replace_dict = {
    'Abra State Institute of Sciences and Technology':
    'Abra State Institute of Science and Technology',
    'Mindoro State College of Agriculture and Technology':
    'Mindoro State University'
}

for key, item in replace_dict.items():
    patents['University'] = patents.University.str.replace(key,
                                                           item)

In [23]:
patent_school_insert = """
INSERT INTO school (school_name, school_abbreviation, 
                    campus, city, province, region)
VALUES (?, ?, ?, ?, ?, ?)
"""


def process_patent_school(cur, conn, patents):

    cur.execute('SELECT * FROM school')
    school_table = pd.DataFrame(cur.fetchall(), 
                                columns=['school_id', 'school_name',
                                        'school_abbreviation', 'campus',
                                        'city', 'province', 'region'])
    
    merge = pd.merge(patents, school_table, left_on=['University'], 
                     right_on=['school_name'], 
                     how='left')
    
    merge = merge.iloc[:, 3:]
    merge = merge.groupby(['Patent Filed', 'Patent Type', 'Patent Status',
               'Date of Registration', 'Registration Number',
               'author_id'], 
                dropna=False)['school_id'].first().reset_index()
    
    return merge

patents = process_patent_school(cur, conn, patents)

In [24]:
patent_insert = """
INSERT INTO patent (patent_filed, patent_type, patent_status, 
                    date_register, registration_number, 
                    author_id, school_id)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""

def process_patent(cur, conn, patents):
    col_update = 'Date of Registration'
    patents[col_update] = patents[col_update].astype(str)
    cur.executemany(patent_insert, 
                    patents.values.tolist())
    
    conn.commit()

process_patent(cur, conn, patents)